Introduction:

Python script to process three data files with messy data sets, to include mistakes throughout the data as well as column headers.
The cleaned data is consolidated and exported as a new CSV, then the new data is used to calculate several statistics which are
combined into a results data frame and exported in CSV format.

In [1]:
import pandas as pd
import json
import re

Parse CSV and drop bad lines:

In [2]:
fortune_df = pd.read_csv('fortune500.csv', on_bad_lines='skip')

Function to parse unstructured text file based on the keys provided and using the colons as a delimeter.
Includes a regex to process bad characters in Revenue and Profit.

In [3]:
def parse_unstructured(file_path):
    data, record = [], {}
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            if line:
                key, value = line.split(':', 1)
                key, value = key.strip(), value.strip()

                if key == 'Year':
                    if record:
                        data.append(record)
                        record = {}
                    record[key] = int(value)
                elif key == 'Rank':
                    record[key] = int(value)
                elif key == 'Company':
                    record[key] = value
                elif 'Revenue' in key:
                    record['Revenue'] = float(re.sub(r'[^\d.]', '', value))
                elif 'Profit' in key:
                    record['Profit'] = float(re.sub(r'[^\d.]', '', value))

            if record:
                data.append(record)

            return pd.DataFrame(data)

Convert text file to data frame.

In [4]:
text_df = parse_unstructured('unstructureddata.txt')

Function to parse JSON file and regex to replace missing values with null or replace bad characters
(a JSON decode error while testing revealed a random question mark in several lines).

In [5]:
def parse_json(file_path):
    data = []

    with open(file_path, 'r') as file:
        for line in file:

            line = line.strip()
            if not line:
                continue
            cleaned_data = re.sub(r'(\w+):,', r'\1": null,', line)
            cleaned_data = re.sub(r'(\w+)\?:', r'\1:', cleaned_data)

            try:
                data.append(json.loads(cleaned_data))
            except json.JSONDecodeError as e:
                continue

    return pd.DataFrame(data)

Convert JSON to data frame.

In [6]:
json_df = parse_json('lines.json')

Clean up the columns:

In [9]:
fortune_df.columns = ['Year', 'Rank', 'Company', 'Revenue', 'Profit']
fortune_df.columns = [col.lower().replace(' ', '_') for col in fortune_df.columns]
text_df.columns = [col.lower().replace(' (in millions)', '') for col in text_df.columns]
json_df.columns = [col.lower().replace(' (in millions)', '') for col in json_df.columns]

Combine all data frames into single frame:

In [10]:
combined_df = pd.concat([fortune_df, text_df, json_df], ignore_index=True)

Copy the data frame so we can calculate the difference in records after cleaning.

In [11]:
cleaned_df = combined_df.copy()

Convert relevant columns to numeric:

In [12]:
for col in ['rank', 'revenue', 'profit']:
    cleaned_df[col] = pd.to_numeric(cleaned_df[col], errors='coerce')

Drop null data records:

In [13]:
cleaned_df.dropna(inplace=True)

Display the head and tail of the cleaned data frame to verify data:

In [14]:
print(cleaned_df.head(10))

      year  rank         company  revenue  profit
0   1955.0   1.0  General Motors   9823.5   806.0
1   1955.0   2.0     Exxon Mobil   5661.4   584.8
2   1955.0   3.0      U.S. Steel   3250.4   195.4
4   1955.0   5.0          Esmark   2510.8    19.1
5   1955.0   6.0        Chrysler   2071.6    18.5
6   1955.0   7.0          Armour   2056.1     1.6
7   1955.0   8.0        Gulf Oil   1705.3   182.8
8   1955.0   9.0           Mobil   1703.6   183.8
9   1955.0  10.0          DuPont   1687.7   344.4
10  1955.0  11.0           Amoco   1667.4   132.8


In [15]:
print(cleaned_df.tail(10))

         year   rank                      company  revenue  profit
39716  1998.0  391.0                H.F. Ahmanson   3732.9   413.8
39717  1998.0  392.0  Supermarkets Genl. Holdings   3710.7   -43.8
39718  1998.0  393.0                    Solectron   3694.4   158.1
39719  1998.0  394.0             Harcourt General   3691.6  -115.1
39720  1998.0  395.0                        Mapco   3689.7    96.9
39721  1998.0  396.0    American Family Ins. Grp.   3689.4   251.6
39722  1998.0  397.0                 Baker Hughes   3685.4    97.0
39723  1998.0  398.0          Service Merchandise   3662.8   -91.6
39724  1998.0  399.0             Silicon Graphics   3662.6    78.6
39725  1998.0  400.0                    Brunswick   3657.4   150.5


Export cleaned data to CSV.

In [16]:
cleaned_df.to_csv('cleaned_data.csv', index=False)

Calculate aggregate data volume, removed lines, and unique companies:

In [17]:
good_data = len(cleaned_df)
bad_data = len(combined_df) - good_data
unique_companies = cleaned_df['company'].nunique()

Reduce data set to specified year range:

In [18]:
df_95to98 = cleaned_df[(cleaned_df['year'] >= 1995) & (cleaned_df['year'] <= 1998)]

Determine highest revenue and profit companies in the new year range:

In [19]:
highest_rev_company = df_95to98.loc[df_95to98['revenue'].idxmax(), 'company']
highest_rev = df_95to98['revenue'].max()

highest_prof_company = df_95to98.loc[df_95to98['profit'].idxmax(), 'company']
highest_prof = df_95to98['profit'].max()

Merge the results into a combined data frame:

In [20]:
Results_Combine = pd.DataFrame({
    'Metric': [
        'Aggregate Data Volume',
        'Instances of Missing Data',
        'Unique Companies',
        'Highest Revenue Company from 1995-98',
        'Highest Profit Company from 1995-98'
    ],
    'Value': [
        good_data,
        bad_data,
        unique_companies,
        f"{highest_rev_company} - ${highest_rev:,.2f}",
        f"{highest_prof_company} - ${highest_prof:,.2f}"
    ]
})

Print the results:

In [21]:
print(Results_Combine)

                                 Metric                         Value
0                 Aggregate Data Volume                         37026
1             Instances of Missing Data                          2700
2                      Unique Companies                          2357
3  Highest Revenue Company from 1995-98  General Motors - $178,174.00
4   Highest Profit Company from 1995-98       Exxon Mobil - $8,460.00


Export the results:

In [22]:
Results_Combine.to_csv('Results_Combine.csv', index=False)